# Extract Data from TMDB

**Objective - Financial Data**
- Stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)
- As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001
- Each year should be saved as a separate .csv.gz file
- Create two functions for extracting data from TMDB
- In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.
- Save the final results to 2 separate .csv.gz files



Source: The Movie Database (TMDB)  https://www.themoviedb.org/

# Imports

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [2]:
import json
with open('C:/Users/david/OneDrive/Documents/May Files/.secret/tmdb.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['APIKey', 'APIReadAccessToken'])

In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['APIKey']